Considere o problema do aspirador de pó definido na Figura 2.2 do livro

In [78]:
class Queue(object):
    def __init__(self):
        self.items = []

    def make_queue(self, elements):
        self.items = elements

    def is_empty(self):
        return len(self.items) == 0

    def remove_front(self):
        return self.items.pop(0)

    def queue_front(self, elements):
        self.items.insert(0, elements)

    def queue_back(self, elements):
        self.items.extend(elements)


class Node(object):
    def __init__(self, state, parent, operator, depth, path_cost):
        self.state = state
        self.parent = parent
        self.operator = operator  # Unused atm
        self.depth = depth
        self.path_cost = path_cost

    @property
    def state_hash(self):
        try:
            return ''.join([str(item) for row in self.state for item in row]) + str(self.operator if self.operator is not None else -1)
        except:
            return ''.join([str(item) for item in self.state]) + self.operator


class GeneralSearch(object):
    def __init__(self, initial_state, expand_node, goal_test):
        self.nodes = [Node(initial_state, None, None, 1, 1)]  # Root Node
        self.expand_node = expand_node  # Function that returns expansion of inputted node
        self.goal_test = goal_test
        self.hashes = {self.nodes[0].state_hash: self.nodes[0]}

        self.queue = Queue()
        self.queue.make_queue(self.expand_node(self.nodes[0]))

    def queuing_function(self, nodes):
        raise NotImplementedError

    @staticmethod
    def solution(node):
        order = []

        while node.parent is not None:
            order.append(node.operator)
            node = node.parent

        order = order[::-1]  # Reverse order to go from root to end

        return order

    def find_path(self, steps=None, search_cost=0):
        i = 1

        while not self.queue.is_empty():
            node = self.queue.remove_front()
            #print(i, "-", node.depth, ":", len(self.queue.items))
            # print "Checking " + node.state_hash

            if self.goal_test(node):
                return self.solution(node), node

            # Check for repeated states
            expanded_nodes = [new_node for new_node in self.expand_node(node) if new_node.state_hash not in self.hashes]

            for new_node in expanded_nodes:
                new_node.path_cost += search_cost  # Add search cost
                self.hashes[new_node.state_hash] = new_node

            self.queuing_function(expanded_nodes)

            if steps is not None and i >= steps:
                return None, None

            i += 1

        return False, None

class DepthFirstSearch(GeneralSearch):
    def __init__(self, initial_state, expand_node, goal_test):
        GeneralSearch.__init__(self, initial_state, expand_node, goal_test)

    def queuing_function(self, nodes):
        self.queue.items = nodes + self.queue.items


In [80]:
from IPython.core.display import TextDisplayObject
import numpy as np
import copy
import warnings


class AmbienteAspirador(object):
    def __init__(self, size=(3, 3), dirt=0.25):
        self.size = size
        self.dirt = dirt

        # Layer 0: dirt, layer 1: objects/home, 2: agent position and direction (0-up, 1-right, 2-down, 3-left)
        self.room = np.zeros((3, self.size[0], self.size[1]))

        for row in range(self.size[0]):
            for col in range(self.size[1]):
                if np.random.uniform() < self.dirt:
                    self.room[0][row][col] = 1

        # Set home base
        home_x = np.random.randint(self.size[0])
        home_y = np.random.randint(self.size[1])
        self.room[1][home_x][home_y] = 1

        self.room[2].fill(-1)
        self.room[2][home_x][home_y] = np.random.randint(4)  # Place agent on home square facing random direction


def goal_test(node):
    # Last action must be switch off

    if node.operator != 4:
        return False

    if np.sum(node.state[0]) == 0:
        home_loc = np.where(node.state[1] == 1)
        agent_loc = np.where(node.state[2] > 0)

        if home_loc == agent_loc:
            return True

    return False


def expand_node(node):
    def has_hit_obstacle(state):
        if (facing == 0 and agent_x == 0) or \
                (facing == 1 and agent_y == len(state[0][1]) - 1) or \
                (facing == 2 and agent_x == len(state[0][0]) - 1) or \
                (facing == 3 and agent_y == 0):
            return True

        return False

    def move_forward(state):
        """
        Updates agents position
        :return: Whether agent hit obstacle
        """

        if has_hit_obstacle(state):
            return state

        if facing == 0:
            state[2][agent_x][agent_y] = -1
            state[2][agent_x - 1][agent_y] = facing

        elif facing == 1:
            state[2][agent_x][agent_y] = -1
            state[2][agent_x][agent_y + 1] = facing

        elif facing == 2:
            state[2][agent_x][agent_y] = -1
            state[2][agent_x + 1][agent_y] = facing

        elif facing == 3:
            state[2][agent_x][agent_y] = -1
            state[2][agent_x][agent_y - 1] = facing

        return state

    def take_action(state, action):
        cost = 101  # Default -1 for each action taken

        if action == 0:
            state = move_forward(state)

        elif action == 1:
            state[2][agent_x][agent_y] = (state[2][agent_x][agent_y] + 1) % 4

        elif action == 2:
            state[2][agent_x][agent_y] = (state[2][agent_x][agent_y] - 1) % 4

        elif action == 3:
            # Reward of +100 for sucking up dirt
            if state[0][agent_x][agent_y] == 1:
                state[0][agent_x][agent_y] = 0

        elif action == 4:
            # If not on home base when switching off give reward of -1000
            if state[1][agent_x][agent_y] != 1:
                cost = 1100

        return state, cost

    # Switch Off stops sequence
    if node.operator == 4:
        return []

    agent_x, agent_y = np.where(node.state[2] >= 0)
    agent_x = agent_x[0]
    agent_y = agent_y[0]

    facing = node.state[2][agent_x][agent_y]

    expanded_nodes = []

    for a in range(5):
        new_node = copy.copy(node)
        new_node.parent = node
        new_node.depth = node.depth + 1
        new_state, action_cost = take_action(copy.copy(node.state), a)
        new_node.state = new_state
        new_node.operator = a
        new_node.path_cost = node.path_cost + action_cost
        expanded_nodes.append(new_node)

    return expanded_nodes

def aspirador_buscaDFS_3x3():
    tipo = "Algoritmo de Busca em Profundidade"
    env = AmbienteAspirador((3, 3), 0)
    env.room = np.array([[[1, 1, 1], [0, 0, 0], [0, 0, 0]], [[0, 0, 0], [0, 1, 0], [0, 0, 0]], [[1, -1, -1], [-1, -1, -1], [-1, -1, -1]]])
    return env, tipo

def aspirador_3x3_ARS():
    tipo = "Agente Reativo Simples"
    env = AmbienteAspirador((3, 3), 0.2)
    env.room = np.array([[[1, 0, 0], [0, 1, 0], [0, 0, 0]], [[1, 0, 0], [0, 0, 0], [0, 0, 0]], [[1, -1, -1], [-1, -1, -1], [-1, -1, -1]]])
    return env, tipo


def aspirador_3x3_ARSA():
    tipo = "Agente Reativo Simples Aleatório"
    env = AmbienteAspirador((3, 3), 0.2)
    env.room[1] = [[1, 0, 0], [0, 0, 0], [0, 0, 0]]
    env.room[2] = [[1, -1, -1], [-1, -1, -1], [-1, -1, -1]]  
    return env, tipo


def main():
    warnings.filterwarnings(action='ignore', category=DeprecationWarning)

    env1, tipo1 = aspirador_buscaDFS_3x3()
    env2, tipo2 = aspirador_3x3_ARS()
    env3, tipo3 = aspirador_3x3_ARSA()

    busca1 = DepthFirstSearch(env1.room, expand_node, goal_test)
    busca2 = DepthFirstSearch(env2.room, expand_node, goal_test)
    busca3 = DepthFirstSearch(env3.room, expand_node, goal_test)

    #print(env.room)

    solution1, node1 = busca1.find_path(search_cost=0)
    solution2, node2 = busca2.find_path(search_cost=0)
    solution3, node3 = busca3.find_path(search_cost=0)

    if solution1 is False:
        print("Não há solução para o algoritmo de busca")
    else:
        print("\n",tipo1)
        print("Custo do caminho:",node1.path_cost, "\nProfundidade:", node1.depth)
        print("Solução:", solution1)

    if solution2 is False:
        print("Não há solução para o Agente Reativo Simples")
    else:
        print("\n",tipo2)
        print("Custo do caminho:",node2.path_cost, "\nProfundidade:", node2.depth)
        print("Solução:", solution2)

    if solution3 is False:
        print("Não há solução para o Agente Reativo Simples Aleatório")
    else:
        print("\n",tipo3)
        print("Custo do caminho:",node3.path_cost, "\nProfundidade:", node3.depth)
        print("Solução:", solution3)


if __name__ == "__main__":
    main()


 Algoritmo de Busca em Profundidade
Custo do caminho: 9192 
Profundidade: 92
Solução: [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 2, 0, 1, 3, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 3, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 2, 0, 2, 4]

 Agente Reativo Simples
Custo do caminho: 7677 
Profundidade: 77
Solução: [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 2, 3, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 4]

 Agente Reativo Simples Aleatório
Custo do caminho: 7879 
Profundidade: 79
Solução: [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 0, 0, 3, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 0, 0, 3, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 0, 2, 4]
